# Sheet properties

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Imports

In [ ]:
import sys
sys.path.append('/home/rcendre/classification')
from numpy import array
from sklearn.preprocessing import LabelEncoder
from toolbox.classification.common import Folds, IO, Tools
from toolbox.classification.parameters import Dermatology, Settings
from toolbox.transforms.labels import OrderedEncoder
from toolbox.transforms.images import DWTGGDImageTransform, DWTImageTransform, FourierImageTransform
from toolbox.views.common import Views

# Parameters

In [ ]:
# Advanced parameters
use_unknown = False 
validation = 4
settings = Settings.get_default_dermatology()

# Inputs

In [ ]:
inputs = Dermatology.images(modality='Microscopy', data_type='Full', use_unknown=use_unknown)

In [ ]:
# Transform groups
group_encoder = LabelEncoder().fit(array(inputs['ID_Patient'].tolist()))
Tools.transform(inputs, {'datum': 'ID_Patient'}, group_encoder, 'GroupEncode')
# Transform labels
label_encoder = OrderedEncoder().fit(['Normal', 'Benign', 'Malignant'])
Tools.transform(inputs, {'datum': 'Label'}, label_encoder, 'LabelEncode')
# Make folds
Folds.build_group_folds(inputs, {'datum': 'Datum', 'label_encode': 'LabelEncode', 'group': 'GroupEncode'}, validation)

# Features extraction

In [ ]:
features_file = f'Extraction_Frequency.pickle'

In [ ]:
extractors = []
# Spatial based
# Fourier
extractors.append(('Fourier10', FourierImageTransform(radius_feat=10)))
extractors.append(('Fourier20', FourierImageTransform(radius_feat=20)))
extractors.append(('Fourier30', FourierImageTransform(radius_feat=30)))

# DWT
extractors.append(('DWTDB4Sc1', DWTImageTransform(wavelets='db4', scale=1)))
extractors.append(('DWTHaarSc1', DWTImageTransform(wavelets='haar', scale=1)))
extractors.append(('DWTHaarSc5', DWTImageTransform(wavelets='haar', scale=5)))

## Methods
extractors.append(('WiltgenFourier', FourierImageTransform()))
extractors.append(('WiltgenDWT', DWTImageTransform(wavelets='db4', scale=5)))
extractors.append(('HalimiDWT', DWTGGDImageTransform(wavelets='db4', scale=4)))

In [ ]:
for extractor in extractors: 
    # Decompose model
    extractor_name, extractor_process = extractor
    
    # Start extracting
    print(f'{extractor_name} performed...', end='\r')
    Tools.transform(inputs, {'datum': 'Datum'}, extractor_process, extractor_name)
    print(f'{extractor_name} achieved !', end='\r')
    
    # Save
    IO.save(inputs, features_file)

## Features

In [ ]:
inputs = IO.load(features_file)
for extractor in extractors: 
    extractor_name, extractor_process = extractor
    # PCA + TSNE
    figure = Views.projection(inputs, {'datum': extractor_name, 'label': 'Label'}, settings);
    figure.savefig(f"{extractor_name}.png", bbox_inches='tight', dpi=300)